# RAG - FSI FAQ with FAISS Local VectorStore Test
이 작업(TASK)에서는 앞서 생성한 SageMaker Embedding Vector 모델의 Endpoint를 활용하여 로컬 환경에서 자주 묻는 질문(FAQ) 데이터셋에 대한 FAISS(Facebook AI Similarity Search) 임베딩을 생성합니다. 이를 통해 텍스트 데이터의 유사성을 빠르게 검색할 수 있게 됩니다. 더불어, SageMaker에서 제공하는 Language Model for Learning-to-Match (LLM) Endpoint를 사용하여, 임베딩으로 검색된 정보를 기반으로 프롬프트 엔지니어링을 수행합니다. 이 과정을 통해 Retriever-Answer Generator (RAG) 로직을 테스트해봅니다.

## Install Install packages and Setup env

In [ ]:
install_needed = True  # should only be True once

In [ ]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -qU pip
    !{sys.executable} -m pip install -qU sagemaker
    !{sys.executable} -m pip install -qU langchain
    !{sys.executable} -m pip install -qU faiss-cpu
    
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. SageMaker Endpoint Wrapper

In [ ]:
# 앞선 노트북에서 저장한 변수들을 로드합니다.
%store -r

In [ ]:
print(endpoint_name)
print(embedding_endpoint_name)

In [ ]:
# Version fixed management during code stabilization period of langchain
!pip install -q SQLAlchemy==2.0.1

In [ ]:
import sys
import IPython
%load_ext autoreload
%autoreload 2
sys.path.append('../utils') # src 폴더 경로 설정

import json
import boto3
import numpy as np
from inference_utils import Prompter
from typing import Any, Dict, List, Optional
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

### 1.1 [한국어 Text LLM] SageMaker Kullm endpiont handler
- Kullm Endpoint의 RequestHandler 함수를 만듭니다.

In [ ]:
prompter = Prompter("kullm")
params = {
      'do_sample': False,
      'max_new_tokens': 512, #128
      'temperature': 1.0,  # 0.5 ~ 1.0 default = 1.0 높으면 랜덤하게 자유도. 다음 생성 문장 토큰의 자유도 
      'top_k': 0,
      'top_p': 0.9,
      'return_full_text': False,
      'repetition_penalty': 1.1,
      'presence_penalty': None,
      'eos_token_id': 2
}

class KullmContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        '''
        입력 데이터 전처리 후에 리턴
        '''
        context, question = prompt.split("||SPEPERATOR||") 
        prompt = prompter.generate_prompt(question, context)

        #print ("prompt", prompt)
        payload = {
            'inputs': [prompt],
            'parameters': model_kwargs
        }
                           
        input_str = json.dumps(payload)
        
        return input_str.encode('utf-8')
    

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))              
        generated_text = response_json[0][0]["generated_text"]
        
        return generated_text    

#### 1.1.a 기존에 deploying한 한국어생성 LLM endpoint 이름을 설정합니다. 
- endpoint_name_text

In [ ]:
aws_region = boto3.Session().region_name
LLMTextContentHandler = KullmContentHandler()

# endpoint_name_text = "kullm-polyglot-5-8b-v2-2023-08-23-15-47-39-450-endpoint"
endpoint_name_text = endpoint_name

seperator = "||SPEPERATOR||"

In [ ]:
llm_text = SagemakerEndpoint(
    endpoint_name=endpoint_name_text,
    region_name=aws_region,
    model_kwargs=params,    
    content_handler=LLMTextContentHandler,
)

### 1.2 [한국어 임베딩벡터 모델] SageMaker 임베딩 벡터 모델 KoSimCSE-roberta endpiont handler
- KoSimCSE-roberta

### SagemakerEndpointEmbeddingsJumpStart클래스는 SagemakerEndpointEmbeddings를 상속받아서 작성

매개변수 (Parameters):
* texts: 임베딩을 생성할 텍스트의 리스트입니다.
* chunk_size: 한 번의 요청에 그룹화될 입력 텍스트의 수를 정의합니다. 만약 None이면, 클래스에 지정된 청크 크기를 사용합니다.

Returns:
* 각 텍스트에 대한 임베딩의 리스트를 반환

In [ ]:
class SagemakerEndpointEmbeddingsJumpStart(SagemakerEndpointEmbeddings):
    def embed_documents(self, texts: List[str], chunk_size: int=1) -> List[List[float]]:
        """Compute doc embeddings using a SageMaker Inference Endpoint.

        Args:
            texts: The list of texts to embed.
            chunk_size: The chunk size defines how many input texts will
                be grouped together as request. If None, will use the
                chunk size specified by the class.

        Returns:
            List of embeddings, one for each text.
        """
        results = []
        _chunk_size = len(texts) if chunk_size > len(texts) else chunk_size
        
        print("text size: ", len(texts))
        print("_chunk_size: ", _chunk_size)

        for i in range(0, len(texts), _chunk_size):
            
            #print (i, texts[i : i + _chunk_size])
            response = self._embedding_func(texts[i : i + _chunk_size])
            #print (i, response, len(response[0].shape))
            
            results.extend(response)
        return results

In [ ]:
class KoSimCSERobertaContentHandler(EmbeddingsContentHandler):
    
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        
        response_json = json.loads(output.read().decode("utf-8"))
        ndim = np.array(response_json).ndim    
        
        if ndim == 4:
            # Original shape (1, 1, n, 768)
            emb = response_json[0][0][0]
            emb = np.expand_dims(emb, axis=0).tolist()
        elif ndim == 2:
            # Original shape (n, 1)
            emb = []
            for ele in response_json:
                e = ele[0][0]
                emb.append(e)
        else:
            print(f"Other # of dimension: {ndim}")
            emb = None
        return emb

#### 1.2.a 기존에 deploying한 임베딩 벡터 endpoint 설정

- endpoint_name_emb

In [ ]:
LLMEmbHandler = KoSimCSERobertaContentHandler()

# endpoint_name_emb = "KoSimCSE-roberta-2023-08-23-14-07-12"
endpoint_name_emb = embedding_endpoint_name

In [ ]:
llm_emb = SagemakerEndpointEmbeddingsJumpStart(
    endpoint_name=endpoint_name_emb,
    region_name=aws_region,
    content_handler=LLMEmbHandler,
)

**Now, we can build an QA application. <span style="color:red">LangChain makes it extremly simple with following few lines of code</span>.**

# 2.Vector Store - FAISS

### FAISS(Facebook AI Similarity Search) : 
- 밀집 벡터의 효율적인 유사성 검색과 클러스터링을 위한 라이브러리입니다. 이 라이브러리는 RAM에 들어가지 않을 수도 있는 어떤 크기의 벡터 세트에서도 검색할 수 있는 알고리즘을 포함하고 있습니다.
- CPU(pip install faiss-cpu)와 GPU(pip install faiss-gpu) 모두에서 사용할 수 있습니다.
- 유사성 검색: 기본적인 유사성 검색 외에도, 유사성 점수를 반환하는 similarity_search_with_score와 같은 FAISS 특화 메서드가 있습니다.
- 벡터 기반 검색: similarity_search_by_vector 메서드를 사용하여 임베딩 벡터를 파라미터로 전달할 수 있습니다.
- 저장 및 로딩: FAISS 인덱스를 로컬에 저장하고 로딩하는 기능이 있어, 매번 인덱스를 생성할 필요가 없습니다.
- 벡터 스토어 병합: 두 개의 FAISS 벡터 스토어를 병합할 수 있습니다.
- 필터링 지원: FAISS는 메타데이터를 기반으로 문서를 필터링하는 기능을 제공합니다.
- 직렬화와 역직렬화: FAISS 인덱스를 바이트로 직렬화하고 역직렬화할 수 있습니다, 이는 데이터베이스에 인덱스를 저장할 때 유용합니다.

### Vector Store :
- Vector Store는 비정형 데이터를 임베딩 벡터로 변환하여 저장하고, 쿼리 시에 해당 임베딩 벡터와 '가장 유사한' 임베딩 벡터를 검색하는 역할을 하는 저장소입니다. 이는 텍스트 임베딩 모델과 연계하여 벡터를 생성하고 저장, 검색하는 기능을 제공합니다. 다양한 벡터 저장 옵션들이 있으며, 비동기 작업도 지원합니다.


References
- Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/
- FAISS Vector Store : https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/faiss



In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

### 2.1 Context file load

#### 2.1.a Increase csv field size

In [ ]:
import csv
csv.field_size_limit(100000000)
csv.field_size_limit()

#### 2.1.b FAQ CSV 파일 Load

In [ ]:
loader = CSVLoader(
    file_path="../dataset/fsi_smart_faq_ko.csv",
    source_column="Source",
    encoding="utf-8"
)
context_documents = loader.load()

#### 2.1.c FSI FAQ Context 데이터 확인 
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [ ]:
print(len(context_documents))

context_documents[0:3]

### 2.2 FAISS 벡터 Indexer 생성

로드한 FAQ 컨텍스트 데이터를 Indexer를 생성 빌드합니다.
- Recursively split by character : https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

It takes about 1 mins
vectorstore 생성에 1분정도 소요됩니다.
Document가 부분 절삭 되지 않도록 Chunking 전략을 잘 세웁니다.

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=FAISS,
    embedding=llm_emb,
    #text_splitter=CharacterTextSplitter(chunk_size=700, chunk_overlap=0),
    #text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200), 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
)
# 로버타 모델의 청크 크기에 종속되어있음. 청크 파라미터 확인

### 2.2.a 임베딩 벡터 스토어 파일 생성 시작

In [ ]:
%%time
index = index_creator.from_loaders([loader])

### 2.2.b index save and read index from local

- 로컬 디렉토리에 인덱싱 데이터를 저장한 후 다시 로드해 봅니다.

In [ ]:
# Save
index.vectorstore.save_local("../indexer/fsi_faq_indexer_ko")

In [ ]:
# Read
index_ = FAISS.load_local("../indexer/fsi_faq_indexer_ko", llm_emb)

### 2.2.c 생성한 Indexer document 5개 확인

In [ ]:
index_.index_to_docstore_id
#index_.docstore._dict[:10]
dict(list(index_.docstore._dict.items())[:5])

In [ ]:
len(list(index_.docstore._dict.items()))

## 3. Langchain QnA

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain

### 3.1 Query and Response

filter_and_remove_score
- 이 함수는 주어진 검색 결과 리스트(res)에서 특정 점수(cutoff_score와 variance를 기반으로)를 기준으로 필터링을 수행하고, 임베딩 Document 이외 검색 결과에 추가된 score를 제거합니다.
- 본 핸즈온 데모에서는 카테고리 분류기 대신, 단순 검색 조건 필터링에 조회되지 않는 에러가 큰 (에러 거리 score가 200이상이고, 분산이 1.3이상인 값을 상한값으로 정한 다음 필터링된 문서 리스트만 반환합니다.
- 실제 환경에서는 문장 분류 카테고리 모델의 사용을 권장합니다. 



In [ ]:
prompt_template = ''.join(["{context}", seperator, "{question}"])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain = load_qa_chain(llm=llm_text, chain_type="stuff", prompt=PROMPT, verbose=True)


def filter_and_remove_score(res, cutoff_score = 200, variance=1.3):
    # Get the lowest score
    lowest_score = min(score for doc, score in res)
    print('lowest_score : ', lowest_score)
    # If the lowest score is over 200, return an empty list
    if lowest_score > cutoff_score:
        return []
    # Calculate the upper bound for scores
    upper_bound = lowest_score * variance

    
    # Filter the list and remove the score
    res = [doc for doc, score in res if score <= upper_bound]

    return res


def get_similiar_docs(query, k=1, fetch_k=50, score=True, bank=""):

    #print (query)
    
    if score:         
        similar_docs = index_.similarity_search_with_score(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        similar_docs=filter_and_remove_score(similar_docs)
    else:        
        similar_docs = index_.similarity_search(
            query,
            k=k,
            fetch_k=fetch_k,
            filter=dict(source=bank)
        )
        
    return similar_docs

def get_answer(query, bank="", k=1):
                
    search_query = query
    
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    
    print("similar_docs : ", similar_docs)
    llm_query = ''+query+' Category에 대한 Information을 찾아서 설명해주세요.'
    
    if not similar_docs:
        llm_query = query

    answer = chain.run(input_documents=similar_docs, question=llm_query)
    
    return answer



def get_search_answer (query, bank="", k=1):
    search_query = query
    similar_docs = get_similiar_docs(search_query, k=k, bank=bank)
    llm_query = '검색된 내용을 no. 별로 요약한 문장으로 알려줘.'
    if not similar_docs:
        llm_query = query

    llm_answer = chain.run(input_documents=similar_docs, question=llm_query)
    return llm_answer

In [ ]:
chain.run(input_documents='', question='안녕하세요')

In [ ]:
%%time
question = "안녕하세요. 날씨가 참 좋네요."
response = get_answer(question, bank="신한은행", k=5)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

### 3.1.a FAQ 데이터 응답문장 테스트

In [ ]:
%%time
question = "타기관OTP 등록 방법 알려주세요"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

In [ ]:
%%time
question = "홈페이지 이용자아이디 여러 개 사용할 수 있나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



In [ ]:

%%time
question = "화면에서 미래를함께하는따뜻한금융 안전한금융거래를위해준비중입니다. 라고 나오고 맨날 멈추는데 어떻게 하냐? 아니쓸수가 없자나"
response = get_answer(question, bank="신한은행", k=1)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



In [ ]:

%%time
question = "공동인증서랑 금융인증서 같이 쓸수있나요?"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



In [ ]:

%%time
question = "홈페이지 탈퇴하는방법 알려줘"
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')



In [ ]:

%%time
question = "질문 답변 감사합니다. 좋은 하루 되세요."
response = get_answer(question, bank="신한은행", k=3)

print (f'question: {question}')
print('==========================')
print('\n')
print (f'response: \n {response}')

